# Безопасное обновление системы
## Организация обновления через канал связи с удалённым сервером

Этот ноутбук реализует систему безопасного обновления с использованием Монитора Безопасности и разделённой системы управления, как показано в диаграмме.

Основные этапы:
- Запрос обновления прошивки.
- Сохранение файла с новой версией прошивки в локальном хранилище.
- Создание цифровой подписи.
- Проверка цифровой подписи и использование новой версии прошивки.
- Запуск системы с новой версией прошивки.

### Создаем WPL файл с моим маршрутом

In [ ]:

wpl_file_content =  """QGC WPL 110
0	1	0	16	0	5	0	0	59.869328 29.827365	0	1
1	0	3	16	0	5	0	0	59.868883 29.824322	0	1
2	0	3	16	0	5	0	0	59.869184 29.823516	0	1
3	0	3	16	0	5	0	0	59.870481 29.828145	0	1
4	0	3	16	0	5	0	0	59.870456 29.836566 0	1
5	0	3	16	0	5	0	0	59.869963 29.844628 0	1
6	0	3	16	0	5	0	0	59.869206 29.848098	0	1
7	0	3	16	0	5	0	0	59.870235 29.849917	0	1
8	0	3	16	0	5	0	0	59.870743 29.849644	0	1
9	0	3	16	0	5	0	0	59.871044 29.849363	0	1
"""

from src.mission_type import GeoSpecificSpeedLimit

speed_limits = [
    GeoSpecificSpeedLimit(0, 20),
    GeoSpecificSpeedLimit(2, 60),
    GeoSpecificSpeedLimit(7, 20),
]

wpl_file = "secure_update/secure_update.wpl"
with open(wpl_file, "w") as f:
    f.write(wpl_file_content)

afcs_present = True

car_id = "secure_update"

### Собираем внешние компоненты программы (остались без измненений из предыдущих модулей)

In [ ]:
import math

from geopy import Point as GeoPoint
from multiprocessing import Queue
from src.event_types import Event
from src.communication_gateway import BaseCommunicationGateway
from src.navigation_system import BaseNavigationSystem
from src.safety_block import BaseSafetyBlock
from src.config import LOG_ERROR, LOG_DEBUG, LOG_INFO, SERVOS_QUEUE_NAME, CARGO_BAY_QUEUE_NAME, CONTROL_SYSTEM_QUEUE_NAME, SAFETY_BLOCK_QUEUE_NAME, SECURITY_MONITOR_QUEUE_NAME

class CommunicationGateway(BaseCommunicationGateway):
    """Класс для реализации логики взаимодействия с системой планирования заданий.
    Работает в отдельном процессе, поэтому создаётся как наследник класса Process.
    """
    def _send_mission_to_consumers(self):
        """Метод для отправки сообщения с маршрутным заданием в систему управления."""
        
        # Получаем имя очереди системы управления
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME 

        # Создаём событие с маршрутным заданием
        event_control = Event(
            source=BaseCommunicationGateway.event_source_name,
            destination=control_q_name,
            operation="set_mission",
            parameters=self._mission
        )
        event_safety = Event(
            source=BaseCommunicationGateway.event_source_name,
            destination=safety_q_name,
            operation="set_mission",
            parameters=self._mission
        )

        # Ищем в каталоге нужную очередь
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        # Отправляем событие в найденную очередь
        security_q.put(event_control)
        security_q.put(event_safety)  

class NavigationSystem(BaseNavigationSystem):
    """ класс навигационного блока """
    def _send_position_to_consumers(self):                              
        safety_q_name = SAFETY_BLOCK_QUEUE_NAME # замените на правильное название очереди
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME # замените на правильное название очереди

        event_safety = Event(
            source=BaseNavigationSystem.event_source_name,
            destination=safety_q_name,
            operation="position_update",
            parameters=self._position           # Координаты текущего местоположения
        )                                       # замените на код создания сообщения с координатами для системы управления 
                                                # подсказка, требуемая операция - position_update

        event_control = Event(
            source=BaseNavigationSystem.event_source_name,
            destination=control_q_name,
            operation="position_update",
            parameters=self._position          
        )     
                                                
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)
        
        security_q.put(event_safety)
        security_q.put(event_control)

class SafetyBlock(BaseSafetyBlock):
    """ Класс ограничений безопасности """

    def _set_new_direction(self, direction: float):
        """ установка нового направления перемещения """
        self._log_message(LOG_INFO, f"новые координаты: {direction}")
        self._log_message(LOG_INFO, f"текущие координаты: {self._position}")
        self._log_message(LOG_DEBUG, f"маршрутное задание: {self._mission}")
        self._log_message(LOG_DEBUG, f"состояние маршруте: {self._route}")

        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, "Неизвестный маршрут или позиция! Остановка.")
            self._stop_the_car()
            return
        
        target_direction = self._calculate_bearing(self._position, self._route.next_point())
        tolerance = 1

        if (abs(target_direction - direction) > tolerance):
            self._log_message(LOG_ERROR, "Направление не совпадает с маршрутом! Остановка.")
            self._stop_the_car()
            return
        
        self._direction = direction    
        self._send_direction_to_consumers()

    def _calculate_bearing(self, start: GeoPoint, end: GeoPoint) -> float:
        if not start or not end:
            self._log_message(LOG_ERROR, "Не указана начальная или конечная точка для расчета направления! Остановка.")
            self._stop_the_car()
            return 0

        delta_longitude = end.longitude - start.longitude
        x = math.sin(math.radians(delta_longitude)) * \
            math.cos(math.radians(end.latitude))
        y = math.cos(math.radians(start.latitude)) * math.sin(math.radians(end.latitude)) - \
            math.sin(math.radians(start.latitude)) * math.cos(math.radians(end.latitude)) * \
            math.cos(math.radians(delta_longitude))

        initial_bearing_rad = math.atan2(x, y)

        # Преобразуем радианы в градусы
        initial_bearing_deg = math.degrees(initial_bearing_rad)

        # Нормализуем значение в диапазоне [0, 360]
        compass_bearing = (initial_bearing_deg + 360) % 360

        return compass_bearing

    def _set_new_speed(self, speed: float):
        """ установка новой скорости """
        self._log_message(LOG_INFO, f"новая скорость: {speed}")
        self._log_message(LOG_INFO, f"текущая скорость: {self._speed}")

        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, "Неизвестный маршрут или позиция! Остановка.")
            self._stop_the_car()
            return 
        
        allowed_speed = self._route.calculate_speed()
        
        if speed > allowed_speed:
            self._log_message(LOG_ERROR, f"Скорость {speed} превышает лимит {allowed_speed}. Ограничиваем.")
            self._speed = allowed_speed
        else:
            self._speed = speed
        
        self._send_speed_to_consumers()

    def _stop_the_car(self):
        self._speed = 0
        self._direction = 0    
        
        self._send_speed_to_consumers()  
        self._send_direction_to_consumers()
        self._send_lock_cargo_to_consumers()

    def _send_speed_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем скорость получателям")
        servos_q_name = SERVOS_QUEUE_NAME

        event_speed = Event(source=self.event_source_name,
                            destination=servos_q_name,
                            operation="set_speed",
                            parameters=self._speed)
        
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        security_q.put(event_speed)

    def _send_direction_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем направление получателям")
        servos_q_name = SERVOS_QUEUE_NAME

        event_direction = Event(source=self.event_source_name,
                                destination=servos_q_name,
                                operation="set_direction",
                                parameters=self._direction)
        
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        security_q.put(event_direction)

    def _lock_cargo(self, _):
        self._log_message(LOG_INFO, "Блокировка грузового отсека")
        self._send_lock_cargo_to_consumers()

    def _release_cargo(self, _):
        if not self._mission or not self._position or not self._route:
            self._log_message(LOG_ERROR, "Неизвестный маршрут или позиция! Остановка.")
            self._stop_the_car()
            return 

        if not self._route.route_finished:
            self._log_message(LOG_INFO, "Маршрут не завершён, выгрузка невозможна!")
            return
        
        self._send_release_cargo_to_consumers()
    

    def _send_lock_cargo_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем команду на блокировку грузового отсека")
        event = Event(
            source=self.event_source_name,
            destination=CARGO_BAY_QUEUE_NAME,
            operation="lock_cargo",
            parameters=None
        )
        
        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        security_q.put(event)

    def _send_release_cargo_to_consumers(self):
        self._log_message(LOG_DEBUG, "Отправляем команду на разблокировку грузового отсека")
        event = Event(
            source=self.event_source_name,
            destination=CARGO_BAY_QUEUE_NAME,
            operation="release_cargo",
            parameters=None
        )

        security_q: Queue = self._queues_dir.get_queue(SECURITY_MONITOR_QUEUE_NAME)

        security_q.put(event)

### Собираем новый класс `ControlSystem` исполняющий код из обновляемого скрипта `control_system_impl.py`

In [ ]:
from src.control_system import BaseControlSystem
import importlib.util
import sys
import os

class ControlSystem(BaseControlSystem):

    def __init__(self, queues_dir, log_level):
        super().__init__(queues_dir)

        self.control_system_instance = None
        self.queues_dir = queues_dir
        self.log_level = log_level
        self.impl_class_name = 'ControlSystemImpl'

        self._refresh_control_system_script(script_path='./secure_update/executable/control_system_impl.py')
    
    def _refresh_control_system_script(self, script_path):
        # Проверяем что файлс со скриптом существует
        if not os.path.exists(script_path):
            raise FileNotFoundError(f"{script_path} does not exist")

        # Загружаем новый исполняемый модуль из скрипта
        spec = importlib.util.spec_from_file_location(self.impl_class_name, script_path)
        module = importlib.util.module_from_spec(spec)
        sys.modules[self.impl_class_name] = module
        spec.loader.exec_module(module)
        control_system_class = getattr(module, self.impl_class_name, None)

        # Проверяем что требуемый класс найден и создаем его экземляр
        if control_system_class is None:
            raise ImportError(f'{self.impl_class_name} class not found.')

        control_system_instance = control_system_class()

        if not hasattr(control_system_instance, '_send_speed_and_direction_to_consumers'):
            raise ImportError(f'{self.impl_class_name} class does not have _send_speed_and_direction_to_consumers method.')
        
        if not hasattr(control_system_instance, '_release_cargo'):
            raise ImportError(f'{self.impl_class_name} class does not have _release_cargo method.')
        
        if not hasattr(control_system_instance, '_lock_cargo'):
            raise ImportError(f'{self.impl_class_name} class does not have _lock_cargo method.')
        
        self.control_system_instance = control_system_instance

    def _send_speed_and_direction_to_consumers(self, speed, direction):
       self.control_system_instance._send_speed_and_direction_to_consumers(speed, direction, self._queues_dir)
    
    def _release_cargo(self):
       self.control_system_instance._release_cargo(self._queues_dir)

    def _lock_cargo(self):
       self.control_system_instance._lock_cargo(self._queues_dir)
    
    def _refresh(self, script_name):
        self._refresh_control_system_script(f'./secure_update/executable/{script_name}')

### Собираем `SecurityMonitor` с политиками общения между новыми компонентами

In [ ]:
from src.security_monitory import BaseSecurityMonitor
from src.security_policy_type import SecurityPolicy

from src.config import COMMUNICATION_GATEWAY_QUEUE_NAME, NAVIGATION_QUEUE_NAME, SAFETY_BLOCK_QUEUE_NAME, CONTROL_SYSTEM_QUEUE_NAME, LOG_DEBUG, LOG_ERROR, LOG_INFO

# Очереди для новых компонентов
from secure_update.config import UPDATE_MANAGER_QUEUE_NAME, UPDATE_VARIFIER_QUEUE_NAME, UPDATER_QUEUE_NAME, USER_REQUEST_QUEUE_NAME
    
class SecurityMonitor(BaseSecurityMonitor):
    """ класс монитора безопасности """

    def __init__(self, queues_dir):
        super().__init__(queues_dir)
        self._init_set_security_policies()

    def _init_set_security_policies(self):
        """ инициализация политик безопасности """
        default_policies = [

            # Политики для новых компонентов
             SecurityPolicy(
                source=USER_REQUEST_QUEUE_NAME,
                destination=UPDATE_MANAGER_QUEUE_NAME,
                operation='update_system'
            ),
            SecurityPolicy(
                source=UPDATE_MANAGER_QUEUE_NAME,
                destination=UPDATE_VARIFIER_QUEUE_NAME,
                operation='sign_srcipt'
            ),
            SecurityPolicy(
                source=UPDATE_VARIFIER_QUEUE_NAME,
                destination=UPDATE_MANAGER_QUEUE_NAME,
                operation='sign_is_done'
            ),
            SecurityPolicy(
                source=UPDATE_MANAGER_QUEUE_NAME,
                destination=UPDATER_QUEUE_NAME,
                operation='update_srcipt'
            ),
            SecurityPolicy(
                source=UPDATER_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='refresh_script'
            ),

            # Политики для старых компонентов (без изменений)
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='set_mission'
            ),
            SecurityPolicy(
                source=COMMUNICATION_GATEWAY_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_mission'
            ),

            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_speed'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='set_direction'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='release_cargo'
            ),
            SecurityPolicy(
                source=CONTROL_SYSTEM_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='lock_cargo'
            ),

            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=SAFETY_BLOCK_QUEUE_NAME,
                operation='position_update'
            ),
            SecurityPolicy(
                source=NAVIGATION_QUEUE_NAME,
                destination=CONTROL_SYSTEM_QUEUE_NAME,
                operation='position_update'
            ),

            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_speed'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=SERVOS_QUEUE_NAME,
                operation='set_direction'
            ),
             SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='lock_cargo'
            ),
            SecurityPolicy(
                source=SAFETY_BLOCK_QUEUE_NAME,
                destination=CARGO_BAY_QUEUE_NAME,
                operation='release_cargo'
            )
        ]

        self.set_security_policies(policies=default_policies)        

    def set_security_policies(self, policies):
        """ установка новых политик безопасности """
        self._security_policies = policies
        self._log_message(
            LOG_INFO, f"изменение политик безопасности: {policies}")

    def _check_event(self, event: Event):
        """ проверка входящих событий """
        self._log_message(
            LOG_DEBUG, f"проверка события {event}, по умолчанию выполнение запрещено")

        authorized = False
        request = SecurityPolicy(
            source=event.source,
            destination=event.destination,
            operation=event.operation)

        if request in self._security_policies:
            self._log_message(
                LOG_DEBUG, "событие разрешено политиками, выполняем")
            authorized = True

        if authorized is False:
            self._log_message(LOG_ERROR, f"событие не разрешено политиками безопасности! {event}")
        return authorized

### Создадим компоненты, которые будут обновлять версию программы

Базовый класс, от которого будем наследовать процессы

In [ ]:
import datetime

from abc import abstractmethod
from multiprocessing import Queue, Process
from queue import Empty
from time import sleep
from src.queues_dir import QueuesDirectory
from src.event_types import Event
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO, CRITICALITY_STR

class BaseProcess(Process):
    """ базовый класс для создания процессов """

    def __init__(self, queues_dir: QueuesDirectory, log_prefix, events_q_name, process_name, log_level=DEFAULT_LOG_LEVEL):
        # вызываем конструктор базового класса
        super().__init__()
        self._log_level = log_level
        self._queues_dir = queues_dir
        self._log_prefix = log_prefix
        self._process_name = process_name

        # создаём очередь для сообщений на обработку
        self._events_q = Queue()
        self._events_q_name = events_q_name

        self._queues_dir.register(
            queue=self._events_q, name=self._events_q_name)

        # инициализируем интервал обновления
        self._recalc_interval_sec = 0.1
        self._quit = False
      
        self._log_message(LOG_INFO, f'создана {self._process_name}')

    def _log_message(self, criticality: int, message: str):
        """_log_message печатает сообщение заданного уровня критичности

        Args:
            criticality (int): уровень критичности
            message (str): текст сообщения
        """
        if criticality <= self._log_level:
            print(f"[{CRITICALITY_STR[criticality]}]{self._log_prefix} {message}")

    def _check_events_q(self):
        """_check_events_q
        проверяет входящие события до их полного исчерпания
        """

        while True:
            try:
                event: Event = self._events_q.get_nowait()
                if not isinstance(event, Event):
                    return
                self._process_event(event)
            except Empty:
                break

    def run(self):
        self._log_message(LOG_INFO, f"старт {self._process_name}")

        while self._quit is False:
            sleep(self._recalc_interval_sec)
            try:
                self._check_events_q()
            except Exception as e:
                self._log_message(LOG_ERROR, f"ошибка {self._process_name}: {e}")
    
    @abstractmethod
    def _process_event(self, event: Event):
        pass


Процесс, имитирующий действия пользователя

Запросы от пользователя настроены следующим образом:
1. Спустя 5 секунд после запуска, пользователь отдает команду на обновление на вредоносную версию программы (наша система должна отбить атаку)
2. Спустя еще 5 секунд, пользователь отдает команду на обновление на допустимую версию программы (наша система должна успешно обновиться)

In [ ]:
import datetime

from time import sleep
from abc import abstractmethod
from multiprocessing import Queue, Process
from queue import Empty
from time import sleep
from src.queues_dir import QueuesDirectory
from src.event_types import Event
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO
from secure_update.config import UPDATE_MANAGER_QUEUE_NAME, USER_REQUEST_QUEUE_NAME

class User(BaseProcess):
    """ класс, имитирующий запросы от пользователя """

    def __init__(self, queues_dir: QueuesDirectory, log_level=DEFAULT_LOG_LEVEL):
        super().__init__(queues_dir=queues_dir, log_prefix="[USER]", events_q_name=USER_REQUEST_QUEUE_NAME, process_name="пользователь", log_level=log_level)
    
    def run(self):
        self._log_message(LOG_INFO, f"старт {self._process_name}")

        # сначала просим обновиться до вредоносной версии программы
        sleep(5)
        self._make_update_request("bad_script.py")

        # потом просим обновиться до нормальной версии программы
        sleep(5)
        self._make_update_request("good_script.py")

            
    def _make_update_request(self, script_name):
        update_manager_q_name = UPDATE_MANAGER_QUEUE_NAME 
        
        event_update = Event(
            source=self._events_q_name,
            destination=update_manager_q_name,
            operation="update_system",
            parameters=script_name
        )                      

        update_manager_q: Queue = self._queues_dir.get_queue(update_manager_q_name)

        update_manager_q.put(event_update)   
        
        self._log_message(LOG_INFO, f"отправил запрос на обновление программы: {script_name}") 


Верификатор проверяет хэш файлов на обновление и подписывает их в случае, если файл на обновление допустимый

In [ ]:
import datetime
import os
import shutil
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA

from abc import abstractmethod
from multiprocessing import Queue, Process
from queue import Empty
from time import sleep
from src.queues_dir import QueuesDirectory
from src.event_types import Event
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO
from secure_update.config import UPDATE_MANAGER_QUEUE_NAME, UPDATE_VARIFIER_QUEUE_NAME

class Verifier(BaseProcess):
    """ базовый класс для верификации файлов с обновлениями """

    def __init__(self, queues_dir: QueuesDirectory, files_hash_map, log_level=DEFAULT_LOG_LEVEL):
        super().__init__(queues_dir=queues_dir, log_prefix="[VERIFIER]", events_q_name=UPDATE_VARIFIER_QUEUE_NAME, process_name="верификатор", log_level=log_level)
        self._files_hash_map = files_hash_map

        self._keys_folder = "./secure_update"
        if not (os.path.exists(f'{self._keys_folder}/private.pem') and os.path.exists(self._keys_folder)):
            self._generate_keys()

    def _process_event(self, event: Event):
        if event.operation == 'sign_srcipt':
            script_file_name = event.parameters
            self._verify_file(script_file_name)
            self._make_verifier_response(script_file_name)

    def _make_verifier_response(self, script_name):
        update_manager_q_name = UPDATE_MANAGER_QUEUE_NAME 
        
        event_update = Event(
            source=self._events_q_name,
            destination=update_manager_q_name,
            operation="sign_is_done",
            parameters=script_name
        )                      

        update_manager_q: Queue = self._queues_dir.get_queue(update_manager_q_name)

        update_manager_q.put(event_update)   
        
        self._log_message(LOG_INFO, f"отправил успешный ответ на подпись: {script_name}") 
        
    def _verify_file(self, file_name):
        source_dir = "./secure_update/destination"

        # Полные пути к файлу
        source_file_path = os.path.join(source_dir, file_name)
        source_sign_path = os.path.join(source_dir, f'{file_name}.sign')
        
        # Проверка, существует ли файл в исходной директории
        if not os.path.exists(source_file_path):
            raise ImportError(f"Файл {source_file_path} не найден.")

        file_hash = self._calculate_sha256(source_file_path)
        self._log_message(LOG_INFO, f"хеш файла {file_name}: {file_hash.hexdigest()}")
        if self._files_hash_map[file_name] != file_hash.hexdigest():
            raise ImportError(f"Файл {source_file_path} не верифицирован.")

        self._sign_file(source_file_path, source_sign_path, file_hash)
        self._log_message(LOG_INFO, f"файл {file_name} успешно провалидирован.")

    def _calculate_sha256(self, file_path):
        with open(file_path, "rb") as f:
            file_data = f.read()
        return SHA256.new(file_data)

    def _sign_file(self, file_path, signature_path, file_hash):
        # Load the private key
        with open(f'{self._keys_folder}/private.pem', "rb") as f:
            private_key = RSA.import_key(f.read())

        # Create a signature
        signature = pkcs1_15.new(private_key).sign(file_hash)

        # Save the signature to a file
        with open(signature_path, "wb") as f:
            f.write(signature)

        self._log_message(LOG_INFO, f"подпись сохранена в {signature_path}.")

    def _generate_keys(self):
        key = RSA.generate(2048)
        private_key = key.export_key()
        with open(f'{self._keys_folder}/private.pem', "wb") as f:
            f.write(private_key)

        public_key = key.publickey().export_key()
        with open(f'{self._keys_folder}/public.pem', "wb") as f:
            f.write(public_key)




Модуль обновления проверяет, что новый файл с обновлением подписан верификатором и запускает процесс обновления внутри контейнера

In [ ]:
import datetime
import os
import shutil
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA

from abc import abstractmethod
from multiprocessing import Queue, Process
from queue import Empty
from time import sleep
from src.queues_dir import QueuesDirectory
from src.event_types import Event
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO, CONTROL_SYSTEM_QUEUE_NAME
from secure_update.config import UPDATER_QUEUE_NAME

class Updater(BaseProcess):
    """ базовый класс для загрузки файлов с обновлениями """

    def __init__(self, queues_dir: QueuesDirectory, log_level=DEFAULT_LOG_LEVEL):
        super().__init__(queues_dir=queues_dir, log_prefix="[UPDATER]", events_q_name=UPDATER_QUEUE_NAME, process_name="модуль обновления", log_level=log_level)
        self._keys_folder = "./secure_update"

    def _process_event(self, event: Event):
        if event.operation == 'update_srcipt':
            script_file_name = event.parameters
            self._verify_signature(script_file_name)
            self._download_file(script_file_name)
            self._make_updater_response(script_file_name)

    def _make_updater_response(self, script_name):
        control_q_name = CONTROL_SYSTEM_QUEUE_NAME 
        
        event_update = Event(
            source=self._events_q_name,
            destination=control_q_name,
            operation="refresh_script",
            parameters=script_name
        )                      

        control_q: Queue = self._queues_dir.get_queue(control_q_name)

        control_q.put(event_update)   
        
        self._log_message(LOG_INFO, f"отправил запрос на использование новой версии программы: {script_name}") 

    def _verify_signature(self, file_name):
        file_path = f'./secure_update/destination/{file_name}'
        signature_path = f'./secure_update/destination/{file_name}.sign'
        public_key_path = f'{self._keys_folder}/public.pem'

        # Читаем оригинальный файл
        if not os.path.exists(file_path):
            raise ImportError(f"Файл {file_path} не найден.")
        with open(file_path, "rb") as f:
            file_data = f.read()
        
        # Создаем хеш от файла
        hash_obj = SHA256.new(file_data)
        
        # Загружаем открытую часть ключа
        if not os.path.exists(public_key_path):
            raise ImportError(f"Ключ {public_key_path} не найден.")
        with open(public_key_path, "rb") as f:
            public_key = RSA.import_key(f.read())
        
        # Читаем подпись
        if not os.path.exists(signature_path):
            raise ImportError(f"Подпись {signature_path} не найдена.")
        with open(signature_path, "rb") as f:
            signature = f.read()
    
        pkcs1_15.new(public_key).verify(hash_obj, signature)
        self._log_message(LOG_INFO, f"подпись файла {file_name} успешно проверена.")

    def _download_file(self, file_name):
        source_dir = "./secure_update/destination"
        destination_dir = "./secure_update/executable"
        
        # Полные пути к файлу
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(destination_dir, file_name)
        
        # Проверка, существует ли файл в исходной директории
        if not os.path.exists(source_file_path):
            raise ImportError(f"Файл {source_file_path} не найден.")

        # Перемещаем файл в директорию назначения
        shutil.copy2(source_file_path, destination_file_path)
        self._log_message(LOG_INFO, f"файл {file_name} успешно перемещен в {destination_dir}.")

Менеджер обновлений инициирует процесс обновления на уровне всей системы

In [ ]:
import datetime
import os
import shutil

from abc import abstractmethod
from multiprocessing import Queue, Process
from queue import Empty
from time import sleep
from src.queues_dir import QueuesDirectory
from src.event_types import Event
from src.config import DEFAULT_LOG_LEVEL, LOG_INFO
from secure_update.config import UPDATE_MANAGER_QUEUE_NAME, UPDATE_VARIFIER_QUEUE_NAME, UPDATER_QUEUE_NAME

class UpdateManager(BaseProcess):
    """ базовый класс для запуска процесса обновления"""

    def __init__(self, queues_dir: QueuesDirectory, log_level=DEFAULT_LOG_LEVEL):
        super().__init__(queues_dir=queues_dir, log_prefix="[UPDATE_MANAGER]", events_q_name=UPDATE_MANAGER_QUEUE_NAME, process_name="менеджер обновлений", log_level=log_level)
    
    def _process_event(self, event: Event):
        if event.operation == 'update_system':
            script_file_name = event.parameters
            self._download_file(script_file_name)
            self._make_verifier_request(script_file_name)
        elif event.operation == 'sign_is_done':
            script_file_name = event.parameters
            self._make_updater_request(script_file_name)
    
    def _download_file(self, file_name):
        source_dir = "./secure_update/source"
        destination_dir = "./secure_update/destination"
        
        # Полные пути к файлу
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(destination_dir, file_name)
        
        # Проверка, существует ли файл в исходной директории
        if not os.path.exists(source_file_path):
            raise ImportError(f"Файл {source_file_path} не найден.")

        # Перемещаем файл в директорию назначения
        shutil.copy2(source_file_path, destination_file_path)
        self._log_message(LOG_INFO, f"файл {file_name} успешно перемещен в {destination_dir}.")

    def _make_verifier_request(self, script_name):
        update_verifier_q_name = UPDATE_VARIFIER_QUEUE_NAME 
        
        event_update = Event(
            source=self._events_q_name,
            destination=update_verifier_q_name,
            operation="sign_srcipt",
            parameters=script_name
        )                      

        update_verifier_q: Queue = self._queues_dir.get_queue(update_verifier_q_name)

        update_verifier_q.put(event_update)   
        
        self._log_message(LOG_INFO, f"отправил запрос на подпись программы: {script_name}") 
    
    def _make_updater_request(self, script_name):
        updater_q_name = UPDATER_QUEUE_NAME 
        
        event_update = Event(
            source=self._events_q_name,
            destination=updater_q_name,
            operation="update_srcipt",
            parameters=script_name
        )                      

        updater_q: Queue = self._queues_dir.get_queue(updater_q_name)

        updater_q.put(event_update)   
        
        self._log_message(LOG_INFO, f"отправил запрос на обновление программы: {script_name}") 


### Теперь еще раз запустим систему, но уже с компонентами, отвечающими за обновление и смотрим на логи

In [ ]:
from time import sleep

from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.mission_planner import Mission
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer
from src.wpl_parser import WPLParser

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)

# новые компонениы программы
update_manager = UpdateManager(queues_dir=queues_dir, log_level=LOG_INFO)
user = User(queues_dir=queues_dir, log_level=LOG_INFO)
verifier = Verifier(queues_dir=queues_dir, files_hash_map={'good_script.py': '2367baac16d4c42ebfb541061e8ae3bc97c526c962cfa5a2cab733e23bc02f32'}, log_level=LOG_INFO)
updater = Updater(queues_dir=queues_dir, log_level=LOG_INFO)

communication_gateway = CommunicationGateway(queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)
navigation_system = NavigationSystem(queues_dir=queues_dir, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

safety_block = SafetyBlock(queues_dir=queues_dir, log_level=LOG_INFO)

security = SecurityMonitor(queues_dir=queues_dir)

# соберём все компоненты для запуска
# в зависимости от наличия СУПА используем разный набор компонентов - с передачей телеметрии или без

system_components = SystemComponentsContainer(
components=[
        # вариант компонентов с передачей телеметрии в СУПА
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security,
        update_manager,
        user,
        verifier,
        updater
    ] if afcs_present else [
        # вариант компонентов для конфигурации без СУПА
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system,
        safety_block,
        security,
        update_manager,
        user,
        verifier,
        updater
    ])

system_components.start()

# настройте этот параметр так, чтобы проверить работу ограничителя
sleep(170)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()